In [1]:
from taxcalc import *
import numpy as np
import pandas as pd
from pandas import DataFrame
import copy

In [2]:
# settings
# Change year here if running simulations for any other year
sim_year = 2017
agi_brackets = [(-1e14, 50000), (50000, 100000), (100000, 1e14)]
# list of dictionaries for each elasticity
elasticities = [
    {'lower': -0.5, 'middle': -0.5, 'upper': -0.5},
    {'lower': -1.0, 'middle': -1.0, 'upper': -1.0},
    {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}]

In [3]:
# lists for each output
base_list = list()
std_list = list()
charitable_list = list()
lower_mtr_list = list()
std_mtr_list = list()
std_charitable_list = list()
all_three_list = list()

In [4]:
# Reform dictionaries

# 2: raise standard deduction
std_reform = {
    sim_year: {
        '_STD': [[11000, 22000, 11000, 11000, 22000]]
    }
}

# 3: allow non-itemizers charitable deduction
charitable_reform = {
    sim_year: {
        '_STD_allow_charity_ded_nonitemizers': [True]
    }
}

# 4: lower top MTR to 35%
mtr_reform = {
    sim_year: {
        '_II_rt7': [0.35]
    }
}

In [5]:
# average charitable contributions by AGI level
contribs = pd.Series([342.96, 857.90, 1585.56])
recs = Records()
pol = Policy()

# blow-up contributions to 2013 levels
for year in range(2010, 2014):
    contribs *= recs.gfactors.factor_value('ATXPY', year)
contribs

0     389.598564
1     974.564405
2    1801.177687
dtype: float64

In [6]:
# intitalize calculator - used to assign charitable giving to non-itemizers
calc = Calculator(records=recs, policy=pol)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [7]:
# assign contribution amounts to non-itemizers based on AGI
calc.records.e19800 = np.where((calc.records.c00100 < 50000) & (calc.records.c04470 == 0),
                               contribs[0], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 >= 50000) & (calc.records.c00100 < 100000) &
                               (calc.records.c04470 == 0),
                               contribs[1], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 > 100000) & (calc.records.c04470 == 0),
                               contribs[2], calc.records.e19800)
charitable_array = copy.deepcopy(calc.records.e19800)

In [8]:
# generate output table
# Total charitable giving only consists of cash contributions. Non-cash can be easily added

def table(calc_table):
    
    dictionary = {'Revenue ($m)': [],
                  'Total Giving ($m)': [], 
                  'Total Giving: Itemizers ($m)': [], 
                  'Total Giving: Non-Itemizers ($m)': [], 
                  'Number of Itemizers (m)': [], 
                  'Number of Non-Itemizers (m)': []}
    
    # Loop through each AGI income bin to get values
    for item in agi_brackets:
        
        val = calc_table.records._combined[(calc_table.records.c00100 >= item[0]) &
                                           (calc_table.records.c00100 < item[1])]
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1])]
        
        dictionary['Revenue ($m)'].append(val.sum() / 1000000)
        
        val = (calc_table.records.e19800[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1])] +
               calc_table.records.e20100[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1])])
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1])]
        
        dictionary['Total Giving ($m)'].append(float(val.sum()) / 1000000)
        
        val = (calc_table.records.e19800[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc_table.records.c04470 > 0)] +
               calc_table.records.e20100[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc_table.records.c04470 > 0)])
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1]) &
                                       (calc_table.records.c04470 > 0)]
        
        dictionary['Total Giving: Itemizers ($m)'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                      (calc_table.records.c00100 < item[1]) &
                                      (calc_table.records.c04470 > 0)]
        
        dictionary['Number of Itemizers (m)'].append(float(val.sum()) / 1000000)

        val = (calc_table.records.e19800[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc_table.records.c04470 == 0)] +
               calc_table.records.e20100[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc_table.records.c04470 == 0)])
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1]) &
                                       (calc_table.records.c04470 == 0)]
        
        dictionary['Total Giving: Non-Itemizers ($m)'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                      (calc_table.records.c00100 < item[1]) &
                                      (calc_table.records.c04470 == 0)]
        
        dictionary['Number of Non-Itemizers (m)'].append(float(val.sum()) / 1000000)
    
    # Add total rows
    for key in dictionary.keys():
        dictionary[key].append(sum(dictionary[key]))
        
    df = DataFrame.from_dict(dictionary)
    df.index = ['$0-50K', '$50K-100K', '$100K+', 'All']
    df = df.round(2)
        
    return(df)

# Begin Simulations

In [9]:
# baseline function - no policy changes, but with elasticities
def base(elast):
    """
    elast: dictionary of elasticities
    returns DF to make the table we're interested in
    """
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    # Set calculator
    recs_base = Records()
    pol_base = Policy()
    beh_base = Behavior()
    calc_base = Calculator(records=recs_base, policy=pol_base, behavior=beh_base,
                          verbose=False)
    beh_base.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_base.records.e19800 = copy.deepcopy(charitable_array)
    
    # assuming we're doing the analysis for 2017
    calc_base.advance_to_year(sim_year)
    
    # create calculator with behavioral response
    calc_base_beh = Behavior.response(calc_noref, calc_base)
    
    # Create and append DF with information interested in
    base_list.append(table(calc_base_beh))

In [10]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
def std(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std = Records()
    pol_std = Policy()
    pol_std.implement_reform(std_reform)
    beh_std = Behavior()
    calc_std = Calculator(records=recs_std, policy=pol_std, behavior=beh_std,
                          verbose=False)
    beh_std.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std.records.e19800 = copy.deepcopy(charitable_array)
    calc_std.advance_to_year(sim_year)
    
    # Create calculator with behavioral reform
    calc_std_beh = Behavior.response(calc_noref, calc_std)
    
    # create and append DF with information interested in
    std_list.append(table(calc_std_beh))

In [11]:
# ability for non-itemizers to itemize charitable deductions 
def charitable(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_charitable = Records()
    pol_charitable = Policy()
    pol_charitable.implement_reform(charitable_reform)
    beh_charitable = Behavior()
    calc_charitable = Calculator(records=recs_charitable, policy=pol_charitable, behavior=beh_charitable,
                                 verbose=False)
    beh_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_charitable.records.e19800 = copy.deepcopy(charitable_array)
    calc_charitable.advance_to_year(sim_year)
    
    # Create calculator with behaviroal reform
    calc_charitable_beh = Behavior.response(calc_noref, calc_charitable)
    
    # Create and append DF with information interested in
    charitable_list.append(table(calc_charitable_beh))

In [12]:
# decrease of the highest marginal tax rate to 35% 
def lower_mtr(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_mtr = Records()
    pol_mtr = Policy()
    pol_mtr.implement_reform(mtr_reform)
    beh_mtr = Behavior()
    calc_mtr = Calculator(records=recs_mtr, policy = pol_mtr, behavior=beh_mtr,
                          verbose=False)
    beh_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_mtr.records.e19800 = copy.deepcopy(charitable_array)
    calc_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_mtr_beh = Behavior.response(calc_noref, calc_mtr)
    
    # Create and append DF with information interested
    lower_mtr_list.append(table(calc_mtr_beh))

In [13]:
# standard deduction change and the marginal rate decrease
def std_mtr(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std_mtr = Records()
    pol_std_mtr = Policy()
    pol_std_mtr.implement_reform(std_reform)
    pol_std_mtr.implement_reform(mtr_reform)
    beh_std_mtr = Behavior()
    calc_std_mtr = Calculator(records=recs_std_mtr, policy=pol_std_mtr, behavior=beh_std_mtr,
                              verbose=False)
    calc_std_mtr.records.e19800 = copy.deepcopy(charitable_array)
    beh_std_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_std_mtr_beh = Behavior.response(calc_noref, calc_std_mtr)
    
    # Create and append df with information interested in
    std_mtr_list.append(table(calc_std_mtr_beh))

In [14]:
# standard deduction change AND the non-itemizer deduction change
def std_charitable(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std_charitable = Records()
    pol_std_charitable = Policy()
    pol_std_charitable.implement_reform(std_reform)
    pol_std_charitable.implement_reform(charitable_reform)
    beh_std_charitable = Behavior()
    calc_std_charitable = Calculator(records=recs_std_charitable, policy=pol_std_charitable,
                                     behavior=beh_std_charitable, verbose=False)
    calc_std_charitable.records.e19800 = copy.deepcopy(charitable_array)
    beh_std_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_charitable.advance_to_year(sim_year)
    
    # Calculator with behavioral response
    calc_std_charitable_beh = Behavior.response(calc_noref, calc_std_charitable)
    
    # Create and append DF with information
    std_charitable_list.append(table(calc_std_charitable_beh))

In [15]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
def all_three(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_three = Records()
    pol_three = Policy()
    pol_three.implement_reform(std_reform)
    pol_three.implement_reform(charitable_reform)
    pol_three.implement_reform(mtr_reform)
    beh_three = Behavior()
    calc_three = Calculator(records=recs_three, policy=pol_three, behavior=beh_three,
                            verbose=False)
    calc_three.records.e19800 = copy.deepcopy(charitable_array)
    beh_three.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_three.advance_to_year(sim_year)
    
    # Calculator with behavior reform
    calc_three_beh = Behavior.response(calc_noref, calc_three)
    
    # Create and append DF with information
    all_three_list.append(table(calc_three_beh))

In [16]:
# loop through each branch of elasticites, calling each function
for item in elasticities:
    base(item)
    std(item)
    charitable(item)
    lower_mtr(item)
    std_mtr(item)
    std_charitable(item)
    all_three(item)

taxcalc/behavior.py:306: RuntimeWarning: invalid value encountered in divide
  delta_income * calc.records.e00200 / agi_m_ided,
taxcalc/behavior.py:310: RuntimeWarning: invalid value encountered in divide
  delta_income * other_income / agi_m_ided,
taxcalc/behavior.py:313: RuntimeWarning: invalid value encountered in divide
  delta_income * ided / agi_m_ided,


# Output (In Millions)

Each table is in the form of a DataFrame and is ready to be exported if need be

# baseline function

In [17]:
# baseline function
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
base_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,7.45,100.64,206210.44,58535.34,12907.83,45627.52
$50K-100K,13.99,18.41,477849.84,55283.41,37966.44,17316.98
$100K+,23.85,6.08,2108967.48,170597.52,160133.99,10463.53
All,45.29,125.13,2793027.75,284416.28,211008.26,73408.02


In [18]:
# baseline function
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
base_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,7.45,100.64,206210.44,58535.34,12907.83,45627.52
$50K-100K,13.99,18.41,477849.84,55283.41,37966.44,17316.98
$100K+,23.85,6.08,2108967.48,170597.52,160133.99,10463.53
All,45.29,125.13,2793027.75,284416.28,211008.26,73408.02


In [19]:
# baseline function
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
base_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,7.45,100.64,206210.44,58535.34,12907.83,45627.52
$50K-100K,13.99,18.41,477849.84,55283.41,37966.44,17316.98
$100K+,23.85,6.08,2108967.48,170597.52,160133.99,10463.53
All,45.29,125.13,2793027.75,284416.28,211008.26,73408.02


# increase value of the standard deduction

In [20]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,3.57,104.52,184714.26,58186.17,6960.90,51225.27
$50K-100K,8.11,24.29,453012.48,54004.03,25579.43,28424.60
$100K+,17.73,12.20,2091078.69,168231.83,145738.47,22493.36
All,29.41,141.01,2728805.43,280422.02,178278.79,102143.23


In [21]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,3.57,104.52,184714.26,58535.34,6960.90,51574.44
$50K-100K,8.11,24.29,453012.48,55283.41,25579.43,29703.98
$100K+,17.73,12.20,2091078.69,170597.52,145738.47,24859.05
All,29.41,141.01,2728805.43,284416.28,178278.79,106137.48


In [22]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,3.57,104.52,184714.26,56973.82,6960.90,50012.92
$50K-100K,8.11,24.29,453012.48,51470.85,25579.43,25891.42
$100K+,17.73,12.20,2091078.69,165005.02,145738.47,19266.56
All,29.41,141.01,2728805.43,273449.70,178278.79,95170.90


# ability for non-itemizers to itemize charitable deductions 

In [23]:
# ability for non-itemizers to itemize charitable deductions 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
charitable_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.20,101.89,203486.26,59631.14,6916.85,52714.29
$50K-100K,11.87,20.53,473426.92,56838.38,24204.13,32634.24
$100K+,22.29,7.64,2104540.29,171718.92,143029.44,28689.48
All,40.35,130.07,2781453.47,288188.44,174150.43,114038.01


In [24]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
charitable_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.20,101.89,203634.24,58535.34,6916.85,51618.49
$50K-100K,11.87,20.53,473747.78,55283.41,24204.13,31079.28
$100K+,22.29,7.64,2104892.58,170597.52,143029.44,27568.08
All,40.35,130.07,2782274.60,284416.28,174150.43,110265.85


In [25]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
charitable_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.20,101.89,202977.64,63435.74,6916.85,56518.89
$50K-100K,11.87,20.53,472798.57,59917.20,24204.13,35713.06
$100K+,22.29,7.64,2104116.83,173248.52,143029.44,30219.08
All,40.35,130.07,2779893.04,296601.46,174150.43,122451.03


# decrease of the highest marginal tax rate to 35% # Lower MTR Results

In [26]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
lower_mtr_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,7.45,100.64,206210.44,58535.34,12907.83,45627.52
$50K-100K,13.99,18.41,477849.84,55283.41,37966.44,17316.98
$100K+,23.85,6.08,2084483.73,169693.46,159235.02,10458.45
All,45.29,125.13,2768544.01,283512.22,210109.28,73402.94


In [27]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
lower_mtr_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,7.45,100.64,206210.44,58535.34,12907.83,45627.52
$50K-100K,13.99,18.41,477849.84,55283.41,37966.44,17316.98
$100K+,23.85,6.08,2084180.59,170597.52,160145.81,10451.71
All,45.29,125.13,2768240.87,284416.28,211020.07,73396.20


In [28]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
lower_mtr_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,7.45,100.64,206210.44,58535.34,12907.83,45627.52
$50K-100K,13.99,18.41,477849.84,55283.41,37966.44,17316.98
$100K+,23.85,6.08,2084897.78,168460.33,157992.31,10468.03
All,45.29,125.13,2768958.06,282279.09,208866.57,73412.52


# standard deduction change and the marginal rate decrease

In [29]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_mtr_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,3.57,104.52,184714.26,58186.17,6960.90,51225.27
$50K-100K,8.11,24.29,453012.48,54004.03,25579.43,28424.60
$100K+,17.74,12.19,2066646.04,167334.66,144842.15,22492.51
All,29.41,141.00,2704372.78,279524.86,177382.48,102142.38


In [30]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_mtr_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,3.57,104.52,184714.26,58535.34,6960.90,51574.44
$50K-100K,8.11,24.29,453012.48,55283.41,25579.43,29703.98
$100K+,17.74,12.19,2066344.85,170597.52,145754.96,24842.56
All,29.42,141.00,2704071.59,284416.28,178295.29,106120.99


In [31]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_mtr_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,3.57,104.52,184714.26,56973.82,6960.90,50012.92
$50K-100K,8.11,24.29,453012.48,51470.85,25579.43,25891.42
$100K+,17.74,12.19,2067057.13,162884.12,143590.42,19293.71
All,29.41,141.01,2704783.87,271328.80,176130.75,95198.05


# standard deduction change AND the non-itemizer deduction change

In [32]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_charitable_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,2.79,105.30,182216.23,59228.81,2903.67,56325.13
$50K-100K,6.43,25.97,446558.06,56628.55,13367.30,43261.25
$100K+,15.70,14.23,2083103.27,170544.04,124670.86,45873.17
All,24.92,145.50,2711877.55,286401.39,140941.83,145459.56


In [33]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_charitable_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,2.79,105.30,182321.55,58535.34,2903.67,55631.67
$50K-100K,6.43,25.97,446844.28,55283.41,13367.30,41916.12
$100K+,15.70,14.23,2083338.18,170597.52,124670.86,45926.66
All,24.92,145.50,2712504.01,284416.28,140941.83,143474.45


In [34]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_charitable_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,2.79,105.30,181856.94,61636.52,2903.67,58732.84
$50K-100K,6.43,25.97,446002.06,59291.91,13367.30,45924.62
$100K+,15.70,14.23,2082922.06,170471.09,124670.86,45800.22
All,24.92,145.50,2710781.06,291399.51,140941.83,150457.68


# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)

In [35]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
all_three_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,2.79,105.30,182216.23,59228.81,2903.67,56325.13
$50K-100K,6.43,25.97,446558.06,56628.55,13367.30,43261.25
$100K+,15.71,14.23,2058790.36,169633.93,124146.99,45486.95
All,24.92,145.50,2687564.65,285491.29,140417.96,145073.33


In [36]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
all_three_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,2.79,105.30,182321.55,58535.34,2903.67,55631.67
$50K-100K,6.43,25.97,446844.28,55283.41,13367.30,41916.12
$100K+,15.71,14.23,2058710.78,170597.52,124957.26,45640.26
All,24.92,145.49,2687876.60,284416.28,141228.23,143188.05


In [37]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
all_three_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,2.79,105.30,181856.94,61636.52,2903.67,58732.84
$50K-100K,6.43,25.97,446002.06,59291.91,13367.30,45924.62
$100K+,15.71,14.23,2059031.43,168319.60,123043.98,45275.62
All,24.92,145.50,2686890.43,289248.03,139314.95,149933.08
